In [1]:
"""This cell contains all of the Pre-processing of the training data"""
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers

# Copy images to training, validation and test directories 
import os,shutil, pathlib
original_dir = pathlib.Path("training_data/raw_images")
classes = os.listdir(original_dir)
classes = len(classes)

"""Generatng a Dataset:
Define the desired image size to resize the images from iNaturalist to (PCs with GPUs will be able to run a larger
image size (255,255,3). Note: in the image size below image_size = (200,200) normally you would include the number of
colour channels (,3) at the end but, if it it isn't defined now keras will automatically default the colour channels to 3.
The batch size is also defined in this section, again more powerful machines could run at a higher batch_size, at the 
and complete model training faster but at the detrement of accuracy)"""

image_size = (200,200)
batch_size = 16

"""
Seperate all the training data downloaded from iNaturalist into a training and validation dataset using 
image_dataset_from_directory() provided by keras preprocessing module. This module requires the arguments: 
Path to image directory, 
Validation_split (as a ratio 0.2 = 20% Validation/80% Training), 
Subset is required if validation split is defined in the function. 
Seed is an optional argument for random shuffling and transformation
image_size - preprocesses the images to the defined size before
batch_size - package images in to batches (number = number of images packaged into each batch)
label_mode - labels to be encoded as categorical for categorical entropy loss later when defining the model. 
"""
train_dataset, validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
"training_data/raw_images",
validation_split=0.2,
subset="both",
seed =1227,
image_size = image_size,
batch_size = batch_size,
label_mode = "categorical",
)

"""
Data augmentation adds random rotations and flips when the image is inputted into the first layer of the model,
RandomFlip is set to horizontal due to it unlikely that images of birds captured in the project will be upside down 
"""
data_augmentation = keras.Sequential(
    [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    ]
)
# Apply "data_augmentation" to the training images
train_dataset= train_dataset.map(
    lambda img, label:(data_augmentation(img),label),
    num_parallel_calls=tf.data.AUTOTUNE,
)

# Prefetching samples in GPU memory helps maximise GPU utilisation 
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(tf.data.AUTOTUNE)


Found 42956 files belonging to 17 classes.
Using 34365 files for training.
Using 8591 files for validation.


In [2]:
# Build the model 

def make_model(input_shape, num_classes):
    inputs=keras.Input(shape= input_shape)
    
    # Entry block 
    x = data_augmentation(inputs) # Perform data augmentation for gpu training 
    x = layers.Rescaling(1.0/255)(x) # Scale pixel values
    x = layers.Conv2D(128,3,strides = 2, padding="same")(x) 
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    previous_block_activation = x # Set aside residual
    
    for size in [256,512,728]: # Loop through filter sizes for multiple convolutional blocks
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size,3,padding="same")(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.MaxPooling2D(3,strides=2, padding="same")(x)
        
        # Project residual 
        
        residual = layers.Conv2D(size,1, strides =2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x,residual]) # Add back residual 
        previous_block_activation = x # Set aside next residual
        
    x = layers.SeparableConv2D(1024,3,padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    x = layers.GlobalAveragePooling2D()(x) # Global average pooling to reduce spatial dimensions
    if num_classes ==2: # Check the number of classes/labels for the activation function. 
        activation = "sigmoid"
        units = 1 
    else:
        activation = "softmax"
        units = num_classes 
        
    x = layers.Dropout(0.5)(x) # drop out for regularization 
    
    x = layers.Flatten()(x) # Flatten the output layer for the final dense layer
    
    outputs = layers.Dense(units, activation = activation)(x)
    print(units)
    return keras.Model(inputs,outputs)

"""The model image_size takes the image size defined at the start and adds a 3 into the argument to tell the model to expect
an RGB image with 3 colour channels
"""
model = make_model(input_shape= image_size +(3,),num_classes = classes)
model.summary()
#keras.utils.plot_model(model,show_shapes=True) 

17
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential (Sequential)        (None, 200, 200, 3)  0           ['input_1[0][0]']                
                                                                                                  
 rescaling (Rescaling)          (None, 200, 200, 3)  0           ['sequential[0][0]']             
                                                                                                  
 conv2d (Conv2D)                (None, 100, 100, 12  3584        ['rescaling[0][0]']       

 separable_conv2d_5 (SeparableC  (None, 25, 25, 728)  537264     ['activation_6[0][0]']           
 onv2D)                                                                                           
                                                                                                  
 batch_normalization_6 (BatchNo  (None, 25, 25, 728)  2912       ['separable_conv2d_5[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 13, 13, 728)  0          ['batch_normalization_6[0][0]']  
                                                                                                  
 conv2d_3 (Conv2D)              (None, 13, 13, 728)  373464      ['add_1[0][0]']                  
                                                                                                  
 add_2 (Ad

In [3]:
# Train the model 
epochs = 20
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = "Species_Classification_Model_Uncropped.h5",
        save_best_only = True,
        monitor="val_loss"),

]
model.compile(
    #optimizer = "rmsprop",
    optimizer = keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
history = model.fit(
    train_dataset,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_dataset,
    #steps_per_epoch = ,
    shuffle = True
)

Epoch 1/20
2148/2148 [==============================] - 355s 163ms/step - loss: 2.7794 - accuracy: 0.1122 - val_loss: 2.7950 - val_accuracy: 0.1216
Epoch 2/20
 530/2148 [======>.......................] - ETA: 4:15 - loss: 2.6818 - accuracy: 0.1307

KeyboardInterrupt: 

In [ ]:
# Displaying curves of loss and accuracy during training
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy)+1)
plt.plot(epochs,accuracy,"bo",label="Training Accuracy")
plt.plot(epochs,val_accuracy,"b",label="Validation Accuracy")
plt.ylabel("Accuracy %")
plt.xlabel("Epochs")
plt.title("Training and validation accuracy")
plt.legend()
plt.savefig("Species Classification Accuracy Uncropped Training Images.png")
plt.figure()
plt.plot(epochs,loss,"bo",label="Training Loss")
plt.plot(epochs,val_loss,"b",label="Validation Loss")
plt.title("Training and validation loss")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()
plt.savefig("Species Classification Loss Uncropped Training Images.png")
plt.show()